In [1]:
import pandas as pd
import numpy as np

X_Bundesliga_score = pd.read_csv('data/X_Bundesliga_score.csv', index_col=0)
Y_Bundesliga_score = pd.read_csv('data/Y_Bundesliga_score.csv', index_col=0)['scores']
weight_Bundesliga_score = pd.read_csv('data/weight_Bundesliga_score.csv', index_col=0)['weight']

X_Bundesliga_score.shape, Y_Bundesliga_score.shape, weight_Bundesliga_score.shape

((964823, 46), (964823,), (964823,))

In [2]:
X_LaLiga_score = pd.read_csv('data/X_LaLiga_score.csv', index_col=0)
Y_LaLiga_score = pd.read_csv('data/Y_LaLiga_score.csv', index_col=0)['scores']
weight_LaLiga_score = pd.read_csv('data/weight_LaLiga_score.csv', index_col=0)['weight']

X_LaLiga_score.shape, Y_LaLiga_score.shape, weight_LaLiga_score.shape

((1200447, 46), (1200447,), (1200447,))

In [3]:
X_Ligue1_score = pd.read_csv('data/X_Ligue1_score.csv', index_col=0)
Y_Ligue1_score = pd.read_csv('data/Y_Ligue1_score.csv', index_col=0)['scores']
weight_Ligue1_score = pd.read_csv('data/weight_Ligue1_score.csv', index_col=0)['weight']

X_Ligue1_score.shape, Y_Ligue1_score.shape, weight_Ligue1_score.shape

((1222501, 46), (1222501,), (1222501,))

In [4]:
X_PremierLeague_score = pd.read_csv('data/X_PremierLeague_score.csv', index_col=0)
Y_PremierLeague_score = pd.read_csv('data/Y_PremierLeague_score.csv', index_col=0)['scores']
weight_PremierLeague_score = pd.read_csv('data/weight_PremierLeague_score.csv', index_col=0)['weight']

X_PremierLeague_score.shape, Y_PremierLeague_score.shape, weight_PremierLeague_score.shape

((1223174, 46), (1223174,), (1223174,))

In [5]:
X_SerieA_score = pd.read_csv('data/X_SerieA_score.csv', index_col=0)
Y_SerieA_score = pd.read_csv('data/Y_SerieA_score.csv', index_col=0)['scores']
weight_SerieA_score = pd.read_csv('data/weight_SerieA_score.csv', index_col=0)['weight']

X_SerieA_score.shape, Y_SerieA_score.shape, weight_SerieA_score.shape

((1230678, 46), (1230678,), (1230678,))

### 拟合模型

In [ ]:
#!pip install xgboost

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 20.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.7/291.7 MB 12.1 MB/s eta 0:00:0000:0100:01
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
from sklearn.metrics import brier_score_loss
import xgboost

X_score = pd.concat([X_Bundesliga_score, X_LaLiga_score, X_Ligue1_score, X_PremierLeague_score, X_SerieA_score], ignore_index=True)
Y_score = pd.concat([Y_Bundesliga_score, Y_LaLiga_score, Y_Ligue1_score, Y_PremierLeague_score, Y_SerieA_score], ignore_index=True)
weight_score = pd.concat([weight_Bundesliga_score, weight_LaLiga_score, weight_Ligue1_score, weight_PremierLeague_score, weight_SerieA_score], ignore_index=True)
folds_score = [0] * len(X_Bundesliga_score) + [1] * len(X_LaLiga_score) + [2] * len(X_Ligue1_score) + [3] * len(X_PremierLeague_score) + [4] * len(X_SerieA_score)
folds_score = np.array(folds_score)

del X_Bundesliga_score, X_LaLiga_score, X_Ligue1_score, X_PremierLeague_score, X_SerieA_score
del Y_Bundesliga_score, Y_LaLiga_score, Y_Ligue1_score, Y_PremierLeague_score, Y_SerieA_score
del weight_Bundesliga_score, weight_LaLiga_score, weight_Ligue1_score, weight_PremierLeague_score, weight_SerieA_score

categorical_features = ['period_id_a0', 'period_id_a1', 'period_id_a2', 'actiontype_a0', 'actiontype_a1', 'actiontype_a2', 'bodypart_a0', 'bodypart_a1', 'bodypart_a2']
for col in categorical_features:
    if X_score[col].dtype != 'category':
        # print(col, X_score[col].dtype)
        X_score[col] = X_score[col].astype('category')
        
X_score.shape, Y_score.shape, weight_score.shape, folds_score

((5841623, 46), (5841623,), (5841623,), array([0, 0, 0, ..., 4, 4, 4]))

In [8]:
model_params_list = [
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 200, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.05, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 10, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 12, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.1,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True},
    {'n_estimators': 300, 'learning_rate': 0.1, 'max_depth': 15, 'reg_alpha': 0.2,
     'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True}
]

best_params, best_mean_brier = 0, 1

for model_params in model_params_list:

    models = [xgboost.XGBClassifier(**model_params)] * 5
    briers = np.zeros(5)

    for i in range(5):
        train_indices = (folds_score != i)
        test_indices = (folds_score == i)
        models[i].fit(X_score[train_indices], Y_score[train_indices], sample_weight=weight_score[train_indices])
        briers[i] = brier_score_loss(Y_score[test_indices],
                                     models[i].predict_proba(X_score[test_indices])[:, 1],
                                     sample_weight=weight_score[test_indices])

    print(model_params)
    print("Test brier score per fold:", briers)
    print("Mean test brier score:", briers.mean())
    
    if briers.mean() < best_mean_brier: best_params, best_mean_brier = model_params, briers.mean()
    
print('Best CV brier score: ', best_mean_brier)
print('Best parameters: ', best_params)

model_score = xgboost.XGBClassifier(**best_params)
model_score.fit(X_score, Y_score, sample_weight=weight_score)

[17:13:02] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4876800, 46, 224332800).
[17:14:07] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4641176, 46, 213494096).
[17:15:10] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4619122, 46, 212479612).
[17:16:14] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4618449, 46, 212448654).
[17:17:18] INFO: /workspace/src/data/iterative_dmatrix.cc:53: Finished constructing the `IterativeDMatrix`: (4610945, 46, 212103470).
{'n_estimators': 100, 'learning_rate': 0.05, 'max_depth': 10, 'reg_alpha': 0, 'random_state': 42, 'verbosity': 2, 'n_jobs': -1, 'enable_categorical': True}
Test brier score per fold: [0.00665567 0.00643164 0.00552932 0.00634642 0.0058407 ]
Mean test brier score: 0.006160750376149317
[17:18:21] INFO: /workspace/src/data/iterative

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=300,
              n_jobs=-1, num_parallel_tree=None, ...)

In [9]:
feature_score = pd.DataFrame({'feature': model_score.feature_names_in_, 'importance': model_score.feature_importances_})
feature_score.sort_values('importance', ascending=False)

,feature,importance
40,actiontype_a0,0.492314
19,dist_to_goal_a0,0.239107
43,bodypart_a0,0.029747
31,dx_a0,0.019394
41,actiontype_a1,0.013443
25,mov_d_a0,0.012322
39,goalscore_diff,0.010139
20,angle_to_goal_a0,0.007265
42,actiontype_a2,0.007036
37,goalscore_team,0.005927


### 保存模型

In [10]:
import joblib
joblib.dump(model_score, 'data/model_score.dat')

['data/model_score.dat']